In [1]:
#  colab用マウントコードの用意
#from google.colab import drive#2020_08_19
#drive.mount('/content/drive')

In [2]:
#いろいろとインストール
import requests #クローリングのためのモジュール
from bs4 import BeautifulSoup as bs4#HTMLから特定の情報を抜き出すためのモジュール
import time     #サーバーに負荷をかけないためのモジュール
import os       #ディレクトリ作成のためのモジュール
import pandas as pd

## 決定版

In [17]:
date='20200814'#日付を入力


race_df=pd.DataFrame(index=[], columns=[])
for i in range(12):
    rno=i+1
    #まず初めに１ページの情報を抜き出す機能
    url='http://www.boatrace.jp/owpc/pc/race/racelist?rno={}&jcd=21&hd={}'.format(rno,date)
    response=requests.get(url)#対象のURLをget
    response.encoding = response.apparent_encoding
    start_page=bs4(response.text, 'html.parser')
    racers_div=start_page.select_one(".is-tableFixed__3rdadd")
    racers_sep_row=racers_div.find_all('tbody') 
    index=0
    race_racers_data=[0]*6
    if len(racers_sep_row)==6:
        for racer_html in racers_sep_row:
            racer_row_ex_td=racer_html.find_all('td')
            #選手の登録ID
            racer_ID_div=racer_row_ex_td[2].find_all('div')
            racer_ID_div=racer_ID_div[0]
            racer_ID_div_txt=racer_ID_div.text
            start=(racer_ID_div_txt.find('/'))-35
            end=(racer_ID_div_txt.find('/')-31)
            racer_ID=racer_ID_div_txt[start:end]#選手の登録ID

            #選手のモータ番号
            racer_moter_td=racer_row_ex_td[6]
            racer_moter_td_text=racer_moter_td.text
            racer_moter_split=racer_moter_td_text.split('\r')
            racer_moter_id=racer_moter_split[1][-2:]
            #選手のボート番号
            racer_boat_td=racer_row_ex_td[7]
            racer_boat_td_text=racer_boat_td.text
            racer_boat_split=racer_boat_td_text.split('\r')
            racer_boat_id=racer_boat_split[1][-2:]
            
            racer_data=[racer_ID,racer_moter_id,racer_boat_id]#選手固有のデータを持ったリスト

            race_racers_data[index]=racer_data
            index+=1
    else:
        print('欠場選手等の例外が発生していないか確認してください')
    #race_racers_dataを一行に変換する
    race_row=pd.Series(index=[])#一レースの情報を一行に変換したもの
    race_num=rno#ここにはレース番号を入れる変数の値を代入
    race_row['number_race']=race_num
    for i in range(len(race_racers_data)):
        race_row['racer_{}_ID'.format(i+1)]=int(race_racers_data[i][0])
        race_row['racer_{}_mo'.format(i+1)]=int(race_racers_data[i][1])
        race_row['racer_{}_bo'.format(i+1)]=int(race_racers_data[i][2])
    race_df=race_df.append(race_row,ignore_index=True)

### 作ったｄｆと選手のパラメータをくっつける

In [23]:
#使用するファイルの定義
file='20'
para_file_path="{}.csv".format(file)
#result_file_path="asiya_result_csv/asiya_result_20{0}.csv".format(year)
write_path="{0}_asiya.csv".format(date)
#/////////////////////////////////////////////以下データフレームの作成
para_df=pd.read_csv(para_file_path)
para_df=para_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
#出力用データフレーム
#pred_race_df=pd.DataFrame(columns=['result_com','number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID','racer_1_rank','racer_1_male','racer_1_age','racer_1_doub','racer_1_ave_st','racer_2_rank','racer_2_male','racer_2_age','racer_2_doub','racer_2_ave_st','racer_3_rank','racer_3_male','racer_3_age','racer_3_doub','racer_3_ave_st','racer_4_rank','racer_4_male','racer_4_age','racer_4_doub','racer_4_ave_st','racer_5_rank','racer_5_male','racer_5_age','racer_5_doub','racer_5_ave_st','racer_6_rank','racer_6_male','racer_6_age','racer_6_doub','racer_6_ave_st'])
pred_race_df=pd.DataFrame(columns=[],index=[])
for index,series in race_df.iterrows():
    add_df=pd.DataFrame(columns=['number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID','racer_1_rank','racer_1_male','racer_1_age','racer_1_doub','racer_1_ave_st','racer_2_rank','racer_2_male','racer_2_age','racer_2_doub','racer_2_ave_st','racer_3_rank','racer_3_male','racer_3_age','racer_3_doub','racer_3_ave_st','racer_4_rank','racer_4_male','racer_4_age','racer_4_doub','racer_4_ave_st','racer_5_rank','racer_5_male','racer_5_age','racer_5_doub','racer_5_ave_st','racer_6_rank','racer_6_male','racer_6_age','racer_6_doub','racer_6_ave_st'])
    #///////////////////////////////////////レースに出ているレーサーの成績を検索＆取得
    ID_1=series['racer_1_ID']
    ID_2=series['racer_2_ID']
    ID_3=series['racer_3_ID']
    ID_4=series['racer_4_ID']
    ID_5=series['racer_5_ID']
    ID_6=series['racer_6_ID']
    racer_1_df=para_df[para_df['racer_ID']==ID_1]
    racer_2_df=para_df[para_df['racer_ID']==ID_2]
    racer_3_df=para_df[para_df['racer_ID']==ID_3]
    racer_4_df=para_df[para_df['racer_ID']==ID_4]
    racer_5_df=para_df[para_df['racer_ID']==ID_5]
    racer_6_df=para_df[para_df['racer_ID']==ID_6]

    #追加していくデータフレームを作成
   
    add_df= pd.DataFrame({'number_race':series['number_race'],
                            'racer_1_ID':series['racer_1_ID'],
                            'racer_2_ID':series['racer_2_ID'],
                            'racer_3_ID':series['racer_3_ID'],
                            'racer_4_ID':series['racer_4_ID'],
                            'racer_5_ID':series['racer_5_ID'],
                            'racer_6_ID':series['racer_6_ID'],
                            'racer_1_bo':series['racer_1_bo'],
                            'racer_1_mo':series['racer_1_mo'],
                            'racer_2_bo':series['racer_2_bo'],
                            'racer_2_mo':series['racer_2_mo'],
                            'racer_3_bo':series['racer_3_bo'],
                            'racer_3_mo':series['racer_3_mo'],
                            'racer_4_bo':series['racer_4_bo'],
                            'racer_4_mo':series['racer_4_mo'],
                            'racer_5_bo':series['racer_5_bo'],
                            'racer_5_mo':series['racer_5_mo'],
                            'racer_6_bo':series['racer_6_bo'],
                            'racer_6_mo':series['racer_6_mo'],
                            'racer_1_rank':racer_1_df.iat[0,1],
                            'racer_1_male':racer_1_df.iat[0,2],
                            'racer_1_age':racer_1_df.iat[0,3],
                            'racer_1_doub':racer_1_df.iat[0,4],
                            'racer_1_ave_st':racer_1_df.iat[0,5],
                            'racer_2_rank':racer_2_df.iat[0,1],
                            'racer_2_male':racer_2_df.iat[0,2],
                            'racer_2_age':racer_2_df.iat[0,3],
                            'racer_2_doub':racer_2_df.iat[0,4],
                            'racer_2_ave_st':racer_2_df.iat[0,5],
                            'racer_3_rank':racer_3_df.iat[0,1],
                            'racer_3_male':racer_3_df.iat[0,2],
                            'racer_3_age':racer_3_df.iat[0,3],
                            'racer_3_doub':racer_3_df.iat[0,4],
                            'racer_3_ave_st':racer_3_df.iat[0,5],
                            'racer_4_rank':racer_4_df.iat[0,1],
                            'racer_4_male':racer_4_df.iat[0,2],
                            'racer_4_age':racer_4_df.iat[0,3],
                            'racer_4_doub':racer_4_df.iat[0,4],
                            'racer_4_ave_st':racer_4_df.iat[0,5],
                            'racer_5_rank':racer_5_df.iat[0,1],
                            'racer_5_male':racer_5_df.iat[0,2],
                            'racer_5_age':racer_5_df.iat[0,3],
                            'racer_5_doub':racer_5_df.iat[0,4],
                            'racer_5_ave_st':racer_5_df.iat[0,5],
                            'racer_6_rank':racer_6_df.iat[0,1],
                            'racer_6_male':racer_6_df.iat[0,2],
                            'racer_6_age':racer_6_df.iat[0,3],
                            'racer_6_doub':racer_6_df.iat[0,4],
                            'racer_6_ave_st':racer_6_df.iat[0,5] }, index=[''])
    #//////////////////////////////データフレームにadd_dfを追加していく。
    pred_race_df=pred_race_df.append(add_df)
pred_race_df.to_csv('{}_starts_asiya.csv'.format(date))

### 前処理、クラスタリング、ダミー変数化を行ってモデルに突っ込むだけの状態にする

In [72]:
#まず初めに１ページの情報を抜き出す機能を作る
url='http://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=21&hd=20200814'
response=requests.get(url)#対象のURLをget
response.encoding = response.apparent_encoding
start_page=bs4(response.text, 'html.parser')
racers_div=start_page.select_one(".is-tableFixed__3rdadd")
racers_sep_row=racers_div.find_all('tbody') 
index=0
race_racers_data=[0]*6

racer_html=racers_sep_row[0]
racer_row_ex_td=racer_html.find_all('td')
#選手の登録ID
racer_ID_div=racer_row_ex_td[2].find_all('div')
racer_ID_div=racer_ID_div[0]
racer_ID_div_txt=racer_ID_div.text
start=(racer_ID_div_txt.find('/'))-35
end=(racer_ID_div_txt.find('/')-31)
racer_ID=racer_ID_div_txt[start:end]#選手の登録ID

#選手のモータ番号
racer_moter_td=racer_row_ex_td[6]
racer_moter_td_text=racer_moter_td.text
racer_moter_split=racer_moter_td_text.split('\r')
racer_moter_id=racer_moter_split[1][-2:]
#racer_moter_id=racer_moter_td.text[26:28]
#選手のボート番号
racer_boat_td=racer_row_ex_td[7]
racer_boat_td_text=racer_boat_td.text
racer_boat_id=racer_boat_td_text[26:28]
racer_data=[racer_ID,racer_moter_id,racer_boat_id]#選手固有のデータを持ったリスト

race_racers_data[index]=racer_data


In [ ]:
#まず初めに１ページの情報を抜き出す機能を作る
url='http://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=21&hd=20200814'
response=requests.get(url)#対象のURLをget
response.encoding = response.apparent_encoding
start_page=bs4(response.text, 'html.parser')
racers_div=start_page.select_one(".is-tableFixed__3rdadd")
racers_sep_row=racers_div.find_all('tbody') 
index=0
race_racers_data=[0]*6
if len(racers_sep_row)==6:
    for racer_html in racers_sep_row:
        racer_row_ex_td=racer_html.find_all('td')
        #選手の登録ID
        racer_ID_div=racer_row_ex_td[2].find_all('div')
        racer_ID_div=racer_ID_div[0]
        racer_ID_div_txt=racer_ID_div.text
        start=(racer_ID_div_txt.find('/'))-35
        end=(racer_ID_div_txt.find('/')-31)
        racer_ID=racer_ID_div_txt[start:end]#選手の登録ID

        #選手のモータ番号
        racer_moter_td=racer_row_ex_td[6]
        racer_moter_td_text=racer_moter_td.text
        racer_moter_id=racer_moter_td.text[26:28]
        #選手のボート番号
        racer_boat_td=racer_row_ex_td[7]
        racer_boat_td_text=racer_boat_td.text
        racer_boat_id=racer_boat_td_text[26:28]
        racer_data=[racer_ID,racer_moter_id,racer_boat_id]#選手固有のデータを持ったリスト

        race_racers_data[index]=racer_data
        index+=1
else:
    print('欠場選手等の例外が発生していないか確認してください')
#race_racers_dataを一行に変換する
race_row=pd.Series(index=[])#一レースの情報を一行に変換したもの
race_num=1#ここにはレース番号を入れる変数の値を代入
race_row['number_race']=race_num
for i in range(len(race_racers_data)):
    race_row['racer_{}_ID'.format(i+1)]=race_racers_data[i][0]
    race_row['racer_{}_mo'.format(i+1)]=race_racers_data[i][1]
    race_row['racer_{}_bo'.format(i+1)]=race_racers_data[i][2]

In [12]:
racer_moter_split=racer_moter_td_text.split('\r')
moter_ID=racer_moter_split[1][-2:]
print(moter_ID)

41


In [11]:
racer_moter_split[1][-2:]

'41'

In [7]:
print(len(racers_div.find_all('tbody')))

6
